In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 3673
valuation_date = date.today()
valuation_date = date(2024, 9, 1)


In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

ブロードリーフ(3673)の銘柄情報

終値: 739.0円(2024年08月30日)
予想配当利回り: 0.27%(2024年08月28日)
予想PER: 728.71倍(2024年08月28日)
実績PBR: 2.9倍(2024年08月28日)
自己資本比率: 61.2%(2024年08月28日)
予想ROE: 0.4%(2024年08月28日)
予想ROA: 0.24%(2024年08月28日)
時価総額: 720.5億円(2024年08月28日)

3673(ブロードリーフ)の四季報データ履歴

発行日: 2024年06月14日
【黒字化】
  業務ソフトは整備業など主力の自動車系でクラウド移行進展し、客単価向上続く。非自動車系向けパッケージも利益貢献着実。クラウド開発費大幅減で、後半にかけて採算良化。営業黒字化。増配余地。
【浸透】
  自動車系クラウド比率は24年末30%、28年中に100%目標(24年3月末約17%)。5年分一括払いの長期契約に力。キャッシュ確保とオプション受注増狙う。

発行日: 2024年03月16日
【浮上】
  業務ソフトは整備業など主力の自動車系のクラウド移行が進み、客単価向上。旅行業等の非自動車系はパッケージ安定貢献。クラウドの開発費大幅減も効き、尻上がりに採算改善。営業黒字浮上。増配も。
【拡大】
  部品流通ネットワークにトヨタ自動車傘下の修理部品・用品販社が出品検討。クラウド型業務ソフトは中堅企業向けERP等の他社サービスと機能連携推進。

発行日: 2023年12月15日
【浮上】
  自動車系のクラウド移行で償却費かさむ。電子帳簿低調。営業赤字続く。24年12月期は自動車系クラウド移行進展。ストック売上比率上昇。携帯電話店や旅行業の非自動車系も伸びて3期ぶり黒字化。
【新サービス】
  中古車販売の支払総額表示に対応したサービス提供、プライスボード作成の負担軽減。クラウド対象業種拡大へ自動車のガラス、部品業者向けに提供開始。

発行日: 2023年09月15日
【赤字】
  自動車系はクラウドサービスへの移行順調。反面パッケージが減少。非自動車向けは好調。クラウド償却費重いが販管費抑制効き、営業赤字縮小。24年12月期はクラウド大幅増で利益貢献。営業黒字化。
【不正検知】
  自動車業界向けクラウドサービスで業務の

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3673,2024-06-30,"""四""",2024-08-08,4407,65,58,21,0.2,1.5,2
3673,2024-03-31,"""四""",2024-05-09,4014,-109,-42,-35,-0.4,-2.7,1
3673,2023-12-31,"""四""",2024-02-09,4136,-380,-436,-459,-5.2,-9.2,4
3673,2023-09-30,"""四""",2023-11-09,3859,-411,-383,-202,-2.3,-10.7,3
3673,2023-06-30,"""四""",2023-08-10,3761,-575,-561,-421,-4.8,-15.3,2
3673,2023-03-31,"""四""",2023-05-10,3629,-536,-541,-405,-4.6,-14.8,1
3673,2022-12-31,"""四""",2023-02-10,3766,-1034,-1091,-998,-11.3,-27.5,4
3673,2022-09-30,"""四""",2022-11-09,3547,-465,-456,-324,-3.7,-13.1,3


In [6]:
# 今季決算予想発表の推移
df = KPL.get_settlement_forcast(code, valuation_date)
df

2024-03-31


code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3673,2024-12-31,"""予""",2024-08-08,17600,50,100,90,1.0,2.0,4
3673,2024-12-31,"""予""",2024-06-18,17600,50,50,40,0.5,2.0,4


In [7]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code, valuation_date)
df

ブロードリーフ(3673)の通期決算(評価日：2024年09月01日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2024年12月期(予)""",2024-08-08,17600,50,100,90,1.0,2.0
"""2023年12月期""",2024-02-09,15385,-1902,-1921,-1487,-16.8,1.0


In [8]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3673,2024-06-30,"""四""",2024-08-08,4407,65,58,21,0.2,1.5,2
3673,2024-03-31,"""四""",2024-05-09,4014,-109,-42,-35,-0.4,-2.7,1
3673,2023-12-31,"""四""",2024-02-09,4136,-380,-436,-459,-5.2,-9.2,4
3673,2023-09-30,"""四""",2023-11-09,3859,-411,-383,-202,-2.3,-10.7,3
3673,2023-06-30,"""四""",2023-08-10,3761,-575,-561,-421,-4.8,-15.3,2
3673,2023-03-31,"""四""",2023-05-10,3629,-536,-541,-405,-4.6,-14.8,1
3673,2022-12-31,"""四""",2023-02-10,3766,-1034,-1091,-998,-11.3,-27.5,4
3673,2022-09-30,"""四""",2022-11-09,3547,-465,-456,-324,-3.7,-13.1,3


In [9]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

ブロードリーフ(3673)の2024年12月期第2四半期決算進捗率(評価日：2024-09-01)


In [10]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

ブロードリーフ(3673)の2023年12月期第2四半期決算進捗率(評価日：2023-09-01)


In [11]:
#### valuation_dateに過去日を指定した場合、valuation_dateを含む期の株価推移
#### 答え合わせ

In [12]:
# 株価推移
# 前期算発表翌営業日～今期決算発表日翌営業日
start_date, end_date = KPL.get_current_settlement_period_by_announcement_date(code, valuation_date, "四")

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>start_date)
start_date = df["date"].to_list()[0]

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>end_date)
if not end_date == date(2999, 12, 31):
    end_date = df["date"].to_list()[0]

PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")<=valuation_date)
vline_date = df["date"].to_list()[-1]
PFIG.add_vline(vline_date)
PFIG.fig.show()

In [13]:
# appendix
KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=date(2024,1,1))

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
3673,2023-12-31,"""本""",2024-02-09,15385,-1902,-1921,-1487,-16.8,1.0,4
3673,2023-12-31,"""四""",2024-02-09,4136,-380,-436,-459,-5.2,-9.2,4
3673,2024-03-31,"""四""",2024-05-09,4014,-109,-42,-35,-0.4,-2.7,1
3673,2024-12-31,"""予""",2024-06-18,17600,50,50,40,0.5,2.0,4
3673,2024-12-31,"""予""",2024-08-08,17600,50,100,90,1.0,2.0,4
3673,2024-06-30,"""四""",2024-08-08,4407,65,58,21,0.2,1.5,2
3673,2024-09-30,"""四""",2024-11-12,4537,332,277,175,2.0,7.3,3
3673,2024-12-31,"""予""",2024-11-12,17800,450,350,125,1.4,2.0,4
